In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd


In [2]:
#client_id = str(getpass.getpass('client_id?'))
#client_secret = str(getpass.getpass('client_secret?'))

In [3]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="client_id",
                                                          client_secret="client_secret"))

In [14]:
#generate cat list
cat_list=[item['id'] for item in sp.categories(limit=50)['categories']['items']]


In [15]:
#generate playlist list
playlist_list=[]
for cat in cat_list:
    playlist_list.extend([item['id'] for item in sp.category_playlists(category_id=cat, limit=50)['playlists']['items']])

In [11]:
#generate song list
song_list=[]
for playlist in playlist_list:
    for i in sp.playlist_tracks(playlist)['items']:
        if i is not None:
            if i['track'] is not None:
                song_list.append(i['track']['id'])                       

In [13]:
#get all audio features in 50 song chunks due to max limit

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

        
audio_feats = []
for chunk in list(chunks(song_list, 50)):
        audio_feats.append(sp.audio_features(chunk))

In [57]:
#take only dict form the audio_feats
chunk_list = [song for artist in audio_feats for song in artist if song is not None and type(song) == dict]

In [58]:
#create df
df = pd.DataFrame(chunk_list)

In [61]:
#export
df.to_csv('checkpoint.csv')